In [10]:
import pandas as pd
import re
import datetime
import ftfy
import emoji
from word2number import w2n
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
df = pd.DataFrame(columns = ['Chapter Name', 'Username', 'Date', 'Text', 
                             'Negative', 'Neutral', 'Positive', 'Compound'])

In [3]:
sid = SentimentIntensityAnalyzer()

In [4]:
for line in open("../data/chaptercomments.csv", mode='r', encoding='utf-8'):
    fields = line.split(",")
    chaptername = fields[0]
    username = fields[1]
    try:
        if fields[3].strip() in ['2020', '2019', '2018', '2017', '2016']:
            date = fields[2] + fields[3]
            comment = fields[4:]
        else:
            if 'ago' in fields[2]:
                s = fields[2]
                if s[0] == 'a':
                    date = "Nov 21 2020"
                else:
                    parsed_s = [s.split()[:2]]
                    time_dict = dict((fmt,float(amount)) for amount,fmt in parsed_s)
                    dt = datetime.timedelta(**time_dict)
                    past_time = datetime.datetime.now() - dt
                    past_time = str(past_time).split()[0].split('-')
                    date = f"{['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][int(past_time[1])]} {past_time[2]} 2020"
                comment = fields[3:]
            else:
                date = fields[2] + " 2020"
                comment = fields[3:]
    except: 
        pass
    comment = " ".join(comment)
    comment = ftfy.fix_text(comment)
    comment = emoji.demojize(comment)
    comment = re.sub("[:,_]", " ", comment)
    if "This comment may be offensive." in comment:
        comment = comment.replace('This comment may be offensive.', '').strip()
    comment = comment.replace(",", '')
    comment = comment.replace('\n', ' ')
    sentiments = dict(sid.polarity_scores(comment))
    row = pd.Series(list([chaptername, username, date, comment,
                         sentiments['neg'], 
                          sentiments['neu'], 
                          sentiments['pos'], 
                          sentiments['compound']]), index=df.columns)
    df = df.append(row, ignore_index=True)   

In [8]:
df

,Chapter Name,Username,Date,Text,Negative,Neutral,Positive,Compound
0,QUEEN OF DEATH,blue_eye56,Nov 09 2020,the family tree is just like one branch,0.000,0.737,0.263,0.3612
1,QUEEN OF DEATH,blue_eye56,Nov 09 2020,I'm already hooked,0.000,1.000,0.000,0.0000
2,QUEEN OF DEATH,laurabear14,Oct 05 2020,this family tree is a straight line skull,0.000,0.759,0.241,0.2263
3,QUEEN OF DEATH,NoVel_Heart_Beat,Oct 05 2020,WHY IS NO ONE TALKING ABOUT THE QUOTE THAT IS ...,0.106,0.544,0.349,0.8272
4,QUEEN OF DEATH,zuzypiia,Oct 02 2020,Demeter is basically a yandere but mom version...,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
8390,FINAL NOTE,harrowingkisses,Mar 27 2018,Wow .......,0.000,0.208,0.792,0.5859
8391,FINAL NOTE,chaotichearts-,Mar 27 2018,Oooh...,0.000,1.000,0.000,0.0000
8392,FINAL NOTE,sweet-tooth-for-pie,Mar 27 2018,god everything about this is so perfect the...,0.000,0.521,0.479,0.9486
8393,FINAL NOTE,collaterals,Mar 27 2018,My description in three words,0.000,1.000,0.000,0.0000


In [9]:
df.to_csv(r'../data/cleancomments.csv')

In [2]:
df = pd.DataFrame(columns = ['Comments', 'Scraped Comments', 'Reads', 'Votes', 'Chapter Name', 'Chapter', 'Part'])

In [3]:
def K_to_int(s):
    if 'K' in s:
        return int(float(s.replace('K', '')) * 1000)
    else: 
        return int(s)

In [29]:
for line in open("../data/chapterdata.csv", mode='r'):
    fields = line.strip().split(',')
    total_comments = K_to_int(fields[0])
    scraped_comments = K_to_int(fields[1])
    total_reads = K_to_int(fields[2])
    total_votes = K_to_int(fields[3])
    chapter_name = fields[4]
    if fields[4] not in ['QUEEN OF DEATH', 'PRELUDE', 'PROLOGUE', 'EPILOGUE', 'POSTLUDE', 'FINAL NOTE']:
        if 'PART' in fields[4]:
            part = w2n.word_to_num(fields[4].replace('PART',''))
            chapter = fields[4]
        else:
            chapter = w2n.word_to_num(fields[4])
            if chapter in [1, 14]:
                part = 1
            if chapter in [15, 40]:
                part = 2
            if chapter in [41, 52]:
                part = 3
    else:
        chapter = fields[4]
        part = 'NA'
    row = pd.Series(list([total_comments, scraped_comments, total_reads, total_votes, chapter_name, chapter, part]), index=df.columns)
    df = df.append(row, ignore_index=True)  

In [30]:
df.to_csv(r'../data/cleandata.csv')